# Import data and packages

In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import seaborn as sns
sns.set_style("darkgrid")
sns.set(style="darkgrid", color_codes=True)

# load data
df = pd.read_csv('./data/final_amazon_beauty.csv')

/Users/bea/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# we will be using bokeh but a couple of them will be run using matplotlob.pyplot
import matplotlib.pyplot as plt

# we need to ensure we are working with the current, stable release of bokeh.
# we will use bokeh for data visualization

!pip install -q bokeh==2.1.1

In [3]:
df.head()

,rating,verified,review_time,reviewer_id,product_id,reviewer_name,reviewer_text,summary,vote,title,brand,rank,main_cat,description,also_view,also_buy,price,similar_item,details
0,5,False,2000-06-03,A2XMFX1BR0IJFJ,0061073717,Jonathan Reed (jonathan.reed2@virgin.net),This calender is brilliant and has plenty of g...,Futurama rules,2,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
1,5,False,2000-05-06,ATKPYXA8XFKGJ,0061073717,Gwen Bates,This calender really is great. In addition to...,A great gift for any futurama fan!,NaN,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
2,1,True,2015-02-19,A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN
3,4,True,2014-12-18,A2F5GHSXFQ0W6J,0143026860,Mary K. Byke,My husband wanted to reading about the Negro ...,... to reading about the Negro Baseball and th...,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN
4,4,True,2014-08-10,A1572GUYS7DGSR,0143026860,David G,"This book was very informative, covering all a...",Worth the Read,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN


In [4]:
df['reviewer_id'].value_counts()

A2GJX2KCUSR0EI    27
A1KSC91G9AIY2Z    23
A1EGCED01USBA9    21
A22CW0ZHY3NJH8    21
AENH50GW3OKDA     20
                  ..
A16IV2M475WI77     1
A2N9UUA0D0397P     1
ARWYNIXSU8N7J      1
A2SU8JOOZXUA98     1
A13LU5IT0S68CG     1
Name: reviewer_id, Length: 324038, dtype: int64

In [5]:
example = df[df.reviewer_id == 'A2GJX2KCUSR0EI']
example.groupby('reviewer_id')['title'].value_counts()

reviewer_id     title                                                                                                                                                                        
A2GJX2KCUSR0EI  Whish Coconut Shaving Cream - Smooth, All Natural Shave Cream for Men & Women, Leaves Skin so Soft, Shea Butter and Coconut Oil, Natural and Organic Skin Care - 5 oz. pump      4
                (3 Pack) e.l.f. Essential All Over Color Stick - Golden Peach                                                                                                                    1
                Briogeo - Rosarco Milk Reparative Leave-In Conditioning Spray, Ultra-Lightweight Leave-In Conditioner for Those with Dry, Damaged, and Brittle Hair, 5 oz                        1
                COVERGIRL Smoothers Moisturizing Concealer, 1 Tube (0.14 oz), For Fair Skin Tones, Solid Stick Concealer, Fragrance Free, Moisturizing (packaging may vary)                      1
                Givenchy Poudr

#### Reviewer ID is more unique that reviewer names. We will be using Reviewer ID as our index.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371345 entries, 0 to 371344
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   rating         371345 non-null  int64 
 1   verified       371345 non-null  bool  
 2   review_time    371345 non-null  object
 3   reviewer_id    371345 non-null  object
 4   product_id     371345 non-null  object
 5   reviewer_name  371293 non-null  object
 6   reviewer_text  370946 non-null  object
 7   summary        371134 non-null  object
 8   vote           51899 non-null   object
 9   title          371329 non-null  object
 10  brand          283989 non-null  object
 11  rank           369010 non-null  object
 12  main_cat       371345 non-null  object
 13  description    258425 non-null  object
 14  also_view      217021 non-null  object
 15  also_buy       148452 non-null  object
 16  price          186438 non-null  object
 17  similar_item   10796 non-null   object
 18  deta

## Number of unique values

In [7]:
print('Reviews are taken from: {} to {}\n'. format(sorted(df.review_time.head(1)).pop(), 
                                                   sorted(df.review_time.tail(1)).pop()))
print('Number of reviews: {:,}'.format(len(df)))
print('Number of customers: {:,}'.format(len(df.reviewer_id.unique())))
print('Number of unique products: {:,}'.format(len(df.product_id.unique())))
print('Number of reviewer name: {:,}'.format(len(df.reviewer_name.unique())))
print('Number of unique name of products: {:,}'.format(len(df.title.unique())))
print('Number of product categories: {:,}'.format(len(df.main_cat.unique())))
print('Number of unique amount of votes: {:,}'.format(len(df.vote.unique())))


Reviews are taken from: 2000-06-03 to 2016-12-15

Number of reviews: 371,345
Number of customers: 324,038
Number of unique products: 32,586
Number of reviewer name: 221,127
Number of unique name of products: 32,392
Number of product categories: 1
Number of unique amount of votes: 392


## Missing data inspection

#### ask advice with Ana on if there's something I can do with this.

In [8]:
def missing(dataset):
    columns = dataset.columns
    print('MISSING ROWS per COLUMN')
    for column in columns:
        percentage = (dataset[column].isnull().sum() / len(dataset)) * 100
        print('{}: {}, {:0.2f}%'.format(column, dataset[column].isnull().sum(), percentage))
        
missing(df)

MISSING ROWS per COLUMN
rating: 0, 0.00%
verified: 0, 0.00%
review_time: 0, 0.00%
reviewer_id: 0, 0.00%
product_id: 0, 0.00%
reviewer_name: 52, 0.01%
reviewer_text: 399, 0.11%
summary: 211, 0.06%
vote: 319446, 86.02%
title: 16, 0.00%
brand: 87356, 23.52%
rank: 2335, 0.63%
main_cat: 0, 0.00%
description: 112920, 30.41%
also_view: 154324, 41.56%
also_buy: 222893, 60.02%
price: 184907, 49.79%
similar_item: 360549, 97.09%
details: 6918, 1.86%


## Changing Data types

Many of the columns are of data type "object" but we need them as strings.

In [9]:
# inspect a few rows
df.head(5)

,rating,verified,review_time,reviewer_id,product_id,reviewer_name,reviewer_text,summary,vote,title,brand,rank,main_cat,description,also_view,also_buy,price,similar_item,details
0,5,False,2000-06-03,A2XMFX1BR0IJFJ,0061073717,Jonathan Reed (jonathan.reed2@virgin.net),This calender is brilliant and has plenty of g...,Futurama rules,2,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
1,5,False,2000-05-06,ATKPYXA8XFKGJ,0061073717,Gwen Bates,This calender really is great. In addition to...,A great gift for any futurama fan!,NaN,Workout Headphones by Arena Essentials,HarperCollins,"3,235,148inBeautyamp;PersonalCare(",All Beauty,NaN,NaN,NaN,NaN,NaN,NaN
2,1,True,2015-02-19,A1V6B6TNIC10QE,0143026860,theodore j bigham,great,One Star,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN
3,4,True,2014-12-18,A2F5GHSXFQ0W6J,0143026860,Mary K. Byke,My husband wanted to reading about the Negro ...,... to reading about the Negro Baseball and th...,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN
4,4,True,2014-08-10,A1572GUYS7DGSR,0143026860,David G,"This book was very informative, covering all a...",Worth the Read,NaN,Black Diamond,Swedish Beauty,"1,462,563inBeautyamp;PersonalCare(",All Beauty,['This is the NEW PURPLE BOTTLE that just came...,['B000LXTNMW'],NaN,NaN,NaN,NaN


In [10]:
# Why is description an array?
df.description[4]

"['This is the NEW PURPLE BOTTLE that just came out (not the old black bottle which came out years ago and has been discontinued)\\n\\nBlack Diamond\\nIndoor Lotion\\nTingle\\nNew\\nFull Size Bottle', '', '']"


# Statistical Inference

- Get the confidence interval of the mean/median of the whole dataset. Use bootstrap (check mini-project

- How to get the confidence interval using only the sample size?

- https://www.evanmiller.org/how-not-to-sort-by-average-rating.html ---> this is basically breaking the 1-5 reviews into "good" and "bad".

In [11]:
review_scores = df.rating.values
review_scores[:5].tolist()

[5, 5, 1, 4, 4]

In [12]:
# # We can set 3.0 as the cutoff between good and bad reviews


# pos_neg_review_scores = list(
#     map(lambda sc: [s >= 4 for s in sc], review_scores.tolist())
# )




# Machine learning 

In [13]:
data = df[['rating', 'reviewer_id', 'product_id']]
# len(df['asin'].value_counts()), len(df['reviewerID'].value_counts())
data

,rating,reviewer_id,product_id
0,5,A2XMFX1BR0IJFJ,0061073717
1,5,ATKPYXA8XFKGJ,0061073717
2,1,A1V6B6TNIC10QE,0143026860
3,4,A2F5GHSXFQ0W6J,0143026860
4,4,A1572GUYS7DGSR,0143026860
...,...,...,...
371340,1,A202DCI7TV1022,B01HJEGTYK
371341,5,A3FSOR5IJOFIBE,B01HJEGTYK
371342,5,A1B5DK6CTP2P24,B01HJEGTYK
371343,2,A23OUYS5IRMJS9,B01HJEGTYK


### Numpy Normalized product matrix

## Create a user-product matrix of of the over-all rates

https://colab.research.google.com/github/couturierc/tutorials/blob/master/recommender_system/Movie_recommender_system_CF_v2_TOFILL.ipynb#scrollTo=uouELHsfKtOU

### Ana's code : 

In [14]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

reviewers = data['reviewer_id'].unique().tolist()
products = data['product_id'].unique().tolist()
reviews = data['rating'].tolist()

row = data.reviewer_id.astype(CategoricalDtype(categories=reviewers)).cat.codes

col = data.product_id.astype(CategoricalDtype(categories=products)).cat.codes

sparse_matrix = csr_matrix((reviews, (row, col)), shape=(len(reviews), len(products)))
R = sparse_matrix.todense()
R

matrix([[5, 0, 0, ..., 0, 0, 0],
        [5, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [15]:
row_cat = data.product_id.astype(CategoricalDtype(categories=products)).cat.categories
col_cat = data.reviewer_id.astype(CategoricalDtype(categories=reviewers)).cat.categories

In [16]:
sparsity = float(len(sparse_matrix.nonzero()[0]))
sparsity /= (sparse_matrix.shape[0] * sparse_matrix.shape[1])
sparsity *= 100
print ('percentage of user-items that have a rating: {:.2f}%'.format(sparsity))

percentage of user-items that have a rating: 0.00%


In [17]:
len(col_cat), np.shape(sparse_matrix)

(324038, (371345, 32586))

In [18]:
# sparse matrix to dataframe
c = pd.DataFrame.sparse.from_spmatrix(sparse_matrix)

In [19]:
# Save the movie ids for user 165 for later:
c.loc[1][:50]

user_165 = (c.loc[165][:50] != 0)

In [20]:
c.loc[1].dropna().sort_values(ascending=False)

0        5
10866    0
10853    0
10854    0
10855    0
        ..
21728    0
21729    0
21730    0
21731    0
16292    0
Name: 1, Length: 32586, dtype: Sparse[int64, 0]

In [21]:
# Save the movie ids for user 1 for later:

item_rated_user_1 = c.loc[1].dropna().index
item_rated_user_1

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            32576, 32577, 32578, 32579, 32580, 32581, 32582, 32583, 32584,
            32585],
           dtype='int64', length=32586)

In [22]:
# Applying SVD using the matrix we made:

from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix, csr_matrix

R = sparse_matrix.asfptype()
U, sigma, Vt = svds(R, k=20)

In [23]:
U

array([[ 9.92931982e-20,  3.24261445e-20,  1.29515599e-19, ...,
         1.63689742e-19,  1.85786596e-20, -5.12043609e-21],
       [ 9.92931982e-20,  3.24261445e-20,  1.29515599e-19, ...,
         1.63689742e-19,  1.85786596e-20, -5.12043609e-21],
       [-1.34778978e-23,  7.94687501e-23, -2.04848840e-22, ...,
        -1.26972815e-22, -3.45332288e-23, -1.37832213e-23],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [24]:
sigma

array([181.91588078, 184.63456262, 184.94826855, 185.07759279,
       189.45588847, 190.67361708, 190.88888856, 199.10164653,
       200.25442434, 204.32755201, 205.53554315, 236.81842639,
       250.04748233, 279.37914424, 288.12141772, 298.02384234,
       315.47887901, 335.65748969, 396.12291467, 586.9313654 ])

In [25]:
Vt

array([[ 3.61260192e-18, -2.45184364e-21, -2.29165858e-09, ...,
        -9.27154684e-18, -7.08888057e-20, -2.16282703e-11],
       [ 1.19739740e-18,  1.46726779e-20,  5.22723041e-09, ...,
        -6.52944485e-18,  9.85889610e-18,  4.29919148e-10],
       [ 4.79073715e-18, -3.78864382e-20,  1.44607648e-07, ...,
         9.64047908e-18, -4.49949979e-18,  6.95182342e-10],
       ...,
       [ 1.09887376e-17, -4.26193762e-20,  3.74578877e-11, ...,
        -3.93444706e-18, -1.55966815e-17,  8.35883290e-12],
       [ 1.47188656e-18, -1.36794033e-20,  9.53231233e-13, ...,
         1.14942484e-18, -2.77997089e-18,  4.18814636e-12],
       [-6.01068909e-19, -8.08980488e-21, -4.87495946e-09, ...,
         2.38354837e-18, -1.08397726e-20, -5.78478491e-13]])

In [26]:
# First make sigma a diagonal matrix:
sigma = np.diag(sigma)

In [27]:
# # It crashes here: I need to apply the new sigma as my diagonal in my matrix!!!

# R_after_svd = np.dot(np.dot(U, sigma), Vt)
# R_after_svd

# Approximate SVD with stochastic gradient descend (SGD)

### This time, we do not fill missing values.

#### We inject Σ into U and V, and try to find P and q such that 𝑅ˆ=𝑃𝑄𝑇 is close to 𝑅 for the item-user pairs already rated.


In [28]:
def encode_ids(data):
    '''Takes a rating dataframe and return: 
    - a simplified rating dataframe with ids in range(nb unique id) for users and movies
    - 2 mapping disctionaries
    
    '''

    data_encoded = data.copy()
    
    users = pd.DataFrame(data_encoded.reviewer_id.unique(),columns=['reviewer_id'])  # df of all unique users
    dict_users = users.to_dict()    
    inv_dict_users = {v: k for k, v in dict_users['reviewer_id'].items()}

    items = pd.DataFrame(data_encoded.product_id.unique(),columns=['product_id']) # df of all unique items
    dict_items = items.to_dict()    
    inv_dict_items = {v: k for k, v in dict_items['product_id'].items()}

    data_encoded.reviewer_id = data_encoded.reviewer_id.map(inv_dict_users)
    data_encoded.product_id = data_encoded.product_id.map(inv_dict_items)

    return data_encoded, dict_users, dict_items

In [29]:
# Adapted from http://nicolas-hug.com/blog/matrix_facto_4
def SGD(data,           # dataframe containing 1 user|item|rating per row
        n_factors = 10, # number of factors
        alpha = .01,    # number of factors
        n_epochs = 3,   # number of iteration of the SGD procedure
       ):
    '''Learn the vectors P and Q (ie all the weights p_u and q_i) with SGD.
    '''

    # Encoding userId's and itemId's in data
    data, dict_users, dict_items = encode_ids(data)
    
    ##### FILL HERE (2 lines) ######
    n_users = data.reviewer_id.nunique()  # number of unique users
    n_items = data.product_id.nunique()  # number of unique items
    ################################
    
    # Randomly initialize the user and item factors.
    p = np.random.normal(0, .1, (n_users, n_factors))
    q = np.random.normal(0, .1, (n_items, n_factors))

    # Optimization procedure
    for epoch in range(n_epochs):
        print ('epoch: ', epoch)
        # Loop over the rows in data
        for index in range(data.shape[0]):
            row = data.iloc[[index]]
            u = int(row.reviewer_id)      # current userId = position in the p vector (thanks to the encoding)
            i = int(row.product_id)      # current itemId = position in the q vector
            r_ui = float(row.rating) # rating associated to the couple (user u , item i)
            
            ##### FILL HERE (1 line) ######
            err = r_ui- np.dot((p[u]),(q[i]).transpose())    # difference between the predicted rating (p_u . q_i) and the known ratings r_ui
            ################################
            
            # Update vectors p_u and q_i
            ##### FILL HERE (2 lines) ######
            p_old = p[u]
            p[u] = p[u] + alpha * err * q[i] # cf. update rule above 
            q[i] = q[i] + alpha * err * p_old  # use p_old instead of p[u]
            ################################
            
    return p, q
    
    
def estimate(u, i, p, q):
    '''Estimate rating of user u for item i.'''
    ##### FILL HERE (1 line) ######
    return np.dot(p[u], q[i]).transpose()             #scalar product of p[u] and q[i] /!\ dimensions
    ################################  

In [30]:
# # Applying SGD to the df we created using encode_ids
# p, q = SGD(data)

In [31]:
# Applying encode_ids function
df_ratings_encoded, dict_users, dict_items = encode_ids(data)

In [32]:
# # It crashes here everytime I try to create the user-item matrix filled with predicted ratings:

# df_user_item_filled = pd.DataFrame(np.dot(p, q.transpose()))
# df_user_item_filled.head()

In [33]:
### DO NOT DELETE: Code for creating the matrix

# df_user_item_filled.rename(columns=(dict_items['product_id']), inplace=True)
# df_user_item_filled.rename(index=(dict_users['reviewer_id']), inplace=True)

# # Sort index/rows (userId's) and columns (itemId's)
# df_user_item_filled.sort_index(axis=0, inplace=True)
# df_user_item_filled.sort_index(axis=1, inplace=True)

# df_user_item_filled.head()

## Pivot Table

## Not showing the entire products

In [34]:
# data = df[['rating', 'reviewer_id', 'product_id']]
# data_table = pd.pivot_table(data,index=["reviewer_id"], columns=["product_id"], values=['rating'], fill_value=0)
# data_table

# # data.reviewerID.unique().shape[0]

### Resampling frequent reviewers

#### Applying recommender system on frequent reviewers >4 reviews

In [35]:
# # frequency = df['reviewerID'].value_counts()
# frequent = (df['reviewerID'].value_counts()[df['reviewerID'].value_counts()> 5]).to_frame()
# reviewers = frequent.reviewerID.to_list()
# top = df['reviewerID'].isin(reviewers)
# top